In [1]:
import sys
sys.path.insert(0, "/mnt/arquivos_linux/wile_C/Predictive_Maintenance_free_dataset/")
import torch
from matplotlib.pyplot import figure
import numpy as np
from datasets import DatasetSinteticUnsupervised
from utils.read_dataset import ReadDatasets

figure(figsize=(8, 6), dpi=80)

limiter = 2000

ABS_PATH = '/mnt/arquivos_linux/wile_C/Predictive_Maintenance_free_dataset'

# test_dataset = DatasetSinteticUnsupervised(dir_data=f'{ABS_PATH}/Datasets/sintetic_data/test_compressor_data.h5',
#                                            context=400)

# test_loader = torch.utils.data.DataLoader(
#                 test_dataset, batch_size=1, shuffle=False
#                 )

<Figure size 640x480 with 0 Axes>

In [2]:
data = ReadDatasets.read_h5(f'{ABS_PATH}/Datasets/sintetic_data/test_compressor_data.h5')

data = np.array(data)

In [3]:
def union_vector(data_predict:dict):
    
    x_concat = data_predict['begin']
    
    del data_predict['begin']
    
    keys_predicted = list(data_predict.keys())
    
    for key in keys_predicted:
        x_concat = torch.concat((x_concat, data_predict[key]), dim=1)
        
        del data_predict[key]
    
    return x_concat

In [4]:
data_predict = torch.load(f"{ABS_PATH}/sintetic_generate_data.pt")

data_predict['begin'].shape

torch.Size([1, 400, 4])

In [5]:
vet_predict = union_vector(data_predict)

In [6]:
vet_predict.shape

torch.Size([1, 13000, 4])

In [7]:
frequency = 65 #Hz
time = 10 #sec
slot = 10000

recons = vet_predict.reshape(-1, time, frequency, 4).numpy()

In [8]:
recons.shape

(20, 10, 65, 4)

## Change data avaliable Test

In [21]:
from sklearn import preprocessing

frequency = 65 #Hz
time = 10 #sec
slot = 20 # reconstruction positional recons.shape[0]

factor = 1

#factor = 1 # para achar o threshold
#factor = 2 # Para avaliar o threshold

data = np.ones((slot, time, frequency, 4))
for i in range(len(data)):
  for j in range(len(data[i])):
    data[i][j] = np.transpose(np.array([np.random.normal(1026.84, factor*18, frequency), 
                                        np.random.normal(26.52, factor*12, frequency),
                                        np.random.normal(84.58, factor*5, frequency),
                                        np.random.normal(100, factor*5, frequency)]))

for i in range(len(data)):
  for j in range(len(data[i])):
    data[i][j] = preprocessing.normalize(data[i][j])

print(data.shape)

(20, 10, 65, 4)


## Avaliable model

In [22]:
recons_error = []
for i in range(len(recons)):
    recons_error.append(np.sqrt((np.sum(data[i]-recons[i]))**2))
    
print(recons_error)

[1.229587398024771, 0.5111189724333757, 0.2620396320603131, 0.16281557589014223, 0.3000050486103717, 0.08869179048326897, 0.5435503865538149, 0.3946481800160774, 0.7118414702273923, 0.0905983433964043, 0.6918589878026336, 0.5025464392206366, 0.2502752042159938, 0.5583553493368224, 0.511523509812467, 0.1225960211046872, 0.7277259669637733, 0.3342148770825443, 0.4414303654927644, 0.08327243733853204]


In [23]:
print(np.mean(recons_error) + np.std(recons_error))

0.7024549778940452


In [24]:
np.mean(recons_error)

0.4259347978033393

In [25]:
np.std(recons_error)

0.2765201800907059

In [26]:
count = 0
for i in range(len(recons_error)):
    if recons_error[i] > 0.65:
        count += 1

print(count)  
print(count/len(recons_error))

4
0.2
